In [10]:
import pandas as pd
import numpy as np
import datetime
from data_tools.api import *
from utilscht.Data import *
from joblib import Parallel,delayed
import logging
from reader.reader import BarReader

logging.basicConfig(level=logging.DEBUG,#控制台打印的日志级别
                    filename=r'logging/stock_pool_renew.log',
                    filemode='a',##模式，有w和a，w就是写模式，每次都会重新写日志，覆盖之前的日志
                    #a是追加模式，默认如果不写的话，就是追加模式
                    format='%(asctime)s - %(pathname)s[line:%(lineno)d] - %(levelname)s: %(message)s'#日志格式
                   )

def apply_parallel(df_grouped, func, n_jobs=16, backend='loky', as_index=True, **kwargs):
    """
    This is for parallel between grouped generated by pd.DataFrame.groupby
    :param df_grouped:
    :param func:
    :param n_jobs:
    :param backend:
    :param kwargs:
    :return:
    """

    names = []
    groups = []
    for name, group in df_grouped:
        names.append(name)
        groups.append(group)

    results = Parallel(n_jobs=n_jobs, verbose=5, backend=backend, batch_size='auto') \
        (delayed(func)(group, **kwargs) for group in groups)

    return pd.concat(results, keys=names if as_index else None)

from data_tools.api import trade_days

trade_dates_all = trade_days.copy()

def get_prev_n_trade_date(trade_date, n):
    trade_date=str(trade_date)[0:10].replace('-','')
    pos = np.searchsorted(trade_dates_all, trade_date)
    try:
        assert pos >= n
        return str(trade_dates_all[pos - n])
    except AssertionError:
        return "20201231"

def get_next_n_trade_date(trade_date, n=1):
    trade_date=str(trade_date)[0:10].replace('-','')
    pos = np.searchsorted(trade_dates_all, trade_date, side='right')
    try:
        assert pos + n - 1 < len(trade_dates_all)
        return str(trade_dates_all[pos + n - 1])
    except AssertionError:
        return "20201231"
    
def get_date_stop_pl(df,ratio = 0.1):
    sid = df["sid"].iloc[0]
    date = df["date"].iloc[0]
    date_now = str(datetime.datetime.now().date()).replace('-','')
    
    df=pd.DataFrame(index=[0],columns=["date_{}p_ud".format(int(ratio*100)),"up_down"])
    if date == date_now:
        df.iloc[0]=(str(date)[0:10],0)
        return df
    
    hq_data = get_stk_bar(sid,start=date,end=get_next_n_trade_date(date,21),fields=["adj_close",'adj_high',"adj_low"])
    thr_up= hq_data["adj_close"].values[0]*(1+ratio)
    thr_down= hq_data["adj_close"].values[0]*(1-ratio)
    
    with BarReader('/home/ywang/proj_cross_grid/reader/bar.yaml') as client:
        bars = client.stock_bars(sid[0:6])
        hq_df = pd.DataFrame(bars)
        low = np.nanmin(hq_df["low"])
        high =  np.nanmax(hq_df["high"])
        close = hq_df["close"].dropna().iloc[-1]
    adj_factor = get_stk_bar(sid,start = get_previous_trade_date(date_now),
                             end = get_previous_trade_date(date_now),fields=["adj_factor"]).values[0][0]
    hq_data = pd.concat([hq_data,pd.DataFrame(index = [date_now],columns=["adj_close",'adj_high',"adj_low"],
                                             data =[(close*adj_factor,high*adj_factor,low*adj_factor)])])
    print(hq_data)
    for dt in hq_data.index[1:21]:
        if hq_data.loc[dt,"adj_high"] > thr_up:
            df.iloc[0]=[(str(dt)[0:10], 1)]
            return df
        if hq_data.loc[dt,"adj_low"] < thr_down:
            df.iloc[0]=[(str(dt)[0:10],-1)]
            return df
    
    change_rate = hq_data.loc[dt,"adj_close"]/hq_data["adj_close"].values[0]-1
    df.iloc[0]=(str(dt)[0:10],change_rate)
    return df

def date_filter(df):
    use_dates=[]
    date_list = df["date"].values
    while(1):
        if len(date_list)==0:
            break
        else:
            date =  min(date_list)
            use_dates.append(date)
            dates_rm = get_trade_dates(date,min(get_next_n_trade_date(date,19),"20201231"))
            date_list=list(set(date_list)-set(dates_rm))
    return df[df["date"].isin(use_dates)]


def get_today_stk_data(sid,date):
    try:
        with BarReader('/home/ywang/proj_cross_grid/reader/bar.yaml') as client:
            bars = client.stock_bars(sid[0:6])
            df = pd.DataFrame(bars)
            df["sid"] = df["symbol"].apply(lambda x:str(x)[2:-1])
            df["low"] = np.nanmin(df["low"])
            df= df[df["sid"] != ""]
            return df[["sid","low"]].iloc[0:1]
    except:
        logging.warning("Data Lost for {} on {}".format(sid,date))

def get_today_data(date):
    pre_date = get_previous_trade_date(date)
    df=query_table("DailyBar",start_date=pre_date,end_date=pre_date,fields=["tradable"])
    stk_ls=list(df["sid"])

    results = Parallel(n_jobs=16, verbose=5, backend="loky", batch_size='auto') \
        (delayed(get_today_stk_data)(sid,date) for sid in stk_ls)
    data = pd.concat(results)

    return data

In [5]:
#!/home/ywang/env/dev/bin
#-*-coding:utf-8-*-

import pandas as pd
import numpy as np
import datetime
from data_tools.api import *
from utilscht.Data import *
from joblib import Parallel,delayed
import logging

logging.basicConfig(level=logging.DEBUG,#控制台打印的日志级别
                    filename=r'logging/stock_pool_renew.log',
                    filemode='a',##模式，有w和a，w就是写模式，每次都会重新写日志，覆盖之前的日志
                    #a是追加模式，默认如果不写的话，就是追加模式
                    format='%(asctime)s - %(pathname)s[line:%(lineno)d] - %(levelname)s: %(message)s'#日志格式
                   )

def apply_parallel(df_grouped, func, n_jobs=16, backend='loky', as_index=True, **kwargs):
    """
    This is for parallel between grouped generated by pd.DataFrame.groupby
    :param df_grouped:
    :param func:
    :param n_jobs:
    :param backend:
    :param kwargs:
    :return:
    """

    names = []
    groups = []
    for name, group in df_grouped:
        names.append(name)
        groups.append(group)

    results = Parallel(n_jobs=n_jobs, verbose=5, backend=backend, batch_size='auto') \
        (delayed(func)(group, **kwargs) for group in groups)

    return pd.concat(results, keys=names if as_index else None)

from data_tools.api import trade_days

trade_dates_all = trade_days.copy()

def get_prev_n_trade_date(trade_date, n):
    trade_date=str(trade_date)[0:10].replace('-','')
    pos = np.searchsorted(trade_dates_all, trade_date)
    try:
        assert pos >= n
        return str(trade_dates_all[pos - n])
    except AssertionError:
        return "20201231"

def get_next_n_trade_date(trade_date, n=1):
    trade_date=str(trade_date)[0:10].replace('-','')
    pos = np.searchsorted(trade_dates_all, trade_date, side='right')
    try:
        assert pos + n - 1 < len(trade_dates_all)
        return str(trade_dates_all[pos + n - 1])
    except AssertionError:
        return "20201231"
    
def get_date_stop_pl(df,ratio = 0.1):
    sid = df["sid"].iloc[0]
    date = df["date"].iloc[0]
    hq_data = get_stk_bar(sid,start=date,end=get_next_n_trade_date(date,21),fields=["adj_close",'adj_high',"adj_low"])
    thr_up= hq_data["adj_close"].values[0]*(1+ratio)
    thr_down= hq_data["adj_close"].values[0]*(1-ratio)
    
    df=pd.DataFrame(index=[0],columns=["date_{}p_ud".format(int(ratio*100)),"up_down"])
    if len(hq_data)<=1:
        df.iloc[0]=(str(date)[0:10],0)
        return df
    
    for dt in hq_data.index[1:21]:
        if hq_data.loc[dt,"adj_high"] > thr_up:
            df.iloc[0]=(str(dt)[0:10], 1)
            return df
        if hq_data.loc[dt,"adj_low"] < thr_down:
            df.iloc[0]=(str(dt)[0:10],-1)
            return df
    
    change_rate = hq_data.loc[dt,"adj_close"]/hq_data["adj_close"].values[0]-1
    df.iloc[0]=(str(dt)[0:10],change_rate)
    return df

def date_filter(df):
    use_dates=[]
    date_list = df["date"].values
    while(1):
        if len(date_list)<=1:
            break
        else:
            date =  min(date_list)
            use_dates.append(date)
            dates_rm = get_trade_dates(date,min(get_next_n_trade_date(date,19),"20201231"))
            date_list=list(set(date_list)-set(dates_rm))
            #print(len(date_list))
    return df[df["date"].isin(use_dates)]



#更新白马
date = "str(datetime.datetime.now().date()).replace('-','')"
pre_date = get_previous_trade_date(date)
stock_pool=pd.read_csv(r"/share/xfzhang/to_colleague/to_yzhao/task2/{}/{}/task2_{}.csv".format(pre_date[0:4],pre_date[4:6],pre_date[0:8]))["sid"]
cross_grid_today=pd.read_excel(r'/home/ywang/proj_cross_grid/result/cross_count_b240_baima/crosscount_summary_{}.xlsx'.format(date))
cross_grid_today=cross_grid_today.set_index("sid").loc[stock_pool].reset_index()[['sid','5d_count','10d_count']]
cross_grid_today = cross_grid_today[(cross_grid_today["5d_count"]>=20) & (cross_grid_today["10d_count"]>=40)]
cross_grid_today['date']=date

cross_grid_summary = pd.read_excel(r"result/穿网格五天大于20十天大于40_白马_全部.xlsx",dtype={"date":str})
date_bound = get_prev_n_trade_date(date,21)
cross_grid_summary_keep = cross_grid_summary[cross_grid_summary["date"]<date_bound]
cross_grid_summary_to_renew = cross_grid_summary[cross_grid_summary["date"]>=date_bound]

cross_grid_summary_to_renew=cross_grid_summary_to_renew.drop(['date_10p_ud','up_down'],axis=1)
cross_grid_summary_to_renew = pd.concat([cross_grid_summary_to_renew,cross_grid_today])
cross_grid_summary_to_renew = cross_grid_summary_to_renew.drop_duplicates(["sid","date"])
group = cross_grid_summary_to_renew.groupby(["sid","date"])
df_date_10p_ud=apply_parallel(group,get_date_stop_pl).reset_index().\
                rename(columns={'level_0':'sid',"level_1":"date"}).drop(['level_2'],axis=1)
cross_grid_summary_to_renew=pd.merge(cross_grid_summary_to_renew,df_date_10p_ud,on=["sid","date"])
cross_grid_summary_raw = pd.concat([cross_grid_summary_keep,cross_grid_summary_to_renew])
cross_grid_summary_raw =cross_grid_summary_raw.sort_values(["date","sid"])
cross_grid_summary_raw.to_excel(r"result/穿网格五天大于20十天大于40_白马_全部.xlsx",index=False)


cross_grid_summary_filtered = cross_grid_summary_raw.groupby("sid").apply(date_filter).reset_index(drop=True)
cross_grid_summary_filtered =cross_grid_summary_filtered.sort_values(["date","sid"])
cross_grid_summary_filtered.to_excel(r"result/穿网格五天大于20十天大于40_白马_20天内不重复触发.xlsx",index=False)

#更新底仓

date = str(datetime.datetime.now().date()).replace('-','')
pre_date = get_previous_trade_date(date)
stock_pool=pd.read_csv(r"/share/xfzhang/to_colleague/to_yzhao/task2_v2/{}/{}/task2_v2_{}.csv".format(pre_date[0:4],pre_date[4:6],pre_date[0:8]))["sid"]

df_today_price=query_table("DailyBar",date,date,fields=["pre_close","low"])
df_today_price=df_today_price.set_index("sid").loc[stock_pool.values].reset_index()
df_today_price["intraday_decre"] = df_today_price["low"] / df_today_price["pre_close"]-1
stock_pool = df_today_price[["sid","intraday_decre"]][df_today_price["intraday_decre"]<-0.03]

cross_grid_today=pd.read_excel(r'/home/ywang/proj_cross_grid/result/cross_count_b240/crosscount_summary_{}.xlsx'.format(date))
cross_grid_today=cross_grid_today.merge(stock_pool,on='sid')[['sid','5d_count','10d_count',"intraday_decre"]]
cross_grid_today = cross_grid_today[(cross_grid_today["5d_count"]>=50) & (cross_grid_today["10d_count"]>=100)]
cross_grid_today["date"]=date

cross_grid_summary = pd.read_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_全部.xlsx",dtype={"date":str})
date_bound = get_prev_n_trade_date(date,21)
cross_grid_summary_keep = cross_grid_summary[cross_grid_summary["date"]<date_bound]
cross_grid_summary_to_renew = cross_grid_summary[cross_grid_summary["date"]>=date_bound]


cross_grid_summary_to_renew=cross_grid_summary_to_renew.drop(['date_10p_ud','up_down'],axis=1)
cross_grid_summary_to_renew = pd.concat([cross_grid_summary_to_renew,cross_grid_today])
cross_grid_summary_to_renew = cross_grid_summary_to_renew.drop_duplicates(["sid","date"])
group = cross_grid_summary_to_renew.groupby(["sid","date"])
df_date_10p_ud=apply_parallel(group,get_date_stop_pl).reset_index().\
                rename(columns={'level_0':'sid',"level_1":"date"}).drop(['level_2'],axis=1)
cross_grid_summary_to_renew=pd.merge(cross_grid_summary_to_renew,df_date_10p_ud,on=["sid","date"])
cross_grid_summary_raw = pd.concat([cross_grid_summary_keep,cross_grid_summary_to_renew])
cross_grid_summary_raw =cross_grid_summary_raw.sort_values(["date","sid"])
cross_grid_summary_raw.to_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_全部.xlsx",index=False)

cross_grid_summary_filtered = cross_grid_summary_raw.groupby("sid").apply(date_filter).reset_index(drop=True)
cross_grid_summary_filtered =cross_grid_summary_filtered.sort_values(["date","sid"])
cross_grid_summary_filtered.to_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_20天内不重复触发.xlsx",index=False)

logging.info("finish"+str(date))

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    2.1s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    4.6s
[Parallel(n_jobs=16)]: Done 256 tasks      | elapsed:    8.4s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   12.9s
[Parallel(n_jobs=16)]: Done 616 tasks      | elapsed:   17.9s
[Parallel(n_jobs=16)]: Done 850 tasks      | elapsed:   24.8s
[Parallel(n_jobs=16)]: Done 1120 tasks      | elapsed:   33.3s
[Parallel(n_jobs=16)]: Done 1344 out of 1344 | elapsed:   39.0s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    1.8s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    4.6s
[Parallel(n_jobs=16)]: Done 256 tasks      | elapsed:    8.4s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   13.2s
[Parallel(n_jobs=16)]: Done 616 tasks      | elapsed:   19.0s
[Parallel(n_jobs=16)]: Done 

In [ ]:
%run stock_pool_renew.py

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    2.2s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    4.7s
[Parallel(n_jobs=16)]: Done 256 tasks      | elapsed:    8.5s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   13.4s
[Parallel(n_jobs=16)]: Done 616 tasks      | elapsed:   19.9s
[Parallel(n_jobs=16)]: Done 850 tasks      | elapsed:   27.6s
[Parallel(n_jobs=16)]: Done 1120 tasks      | elapsed:   36.1s
[Parallel(n_jobs=16)]: Done 1195 out of 1195 | elapsed:   38.5s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    2.0s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    4.8s
[Parallel(n_jobs=16)]: Done 256 tasks      | elapsed:    8.8s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   13.8s
[Parallel(n_jobs=16)]: Done 616 tasks      | elapsed:   20.4s
[Parallel(n_jobs=16)]: Done 

In [19]:
import pandas as pd
temp_df = pd.read_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_全部.xlsx")
temp_df=temp_df[temp_df["date"]<20191212]

temp_df_part2 = pd.read_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_全部_2.xlsx")
temp_df_part2=temp_df_part2.sort_values(["date","sid"])

temp_df= pd.concat([temp_df,temp_df_part2])
temp_df.to_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_全部.xlsx")